# DB에서 상장 기업 정보를 받아 네이버 증권에서 주가 수집하기

In [7]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql
import time

In [8]:
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path=".env_db")

True

In [11]:
# mysql에서 테이블 불러오기
engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
conn = engine.connect()
data = pd.read_sql('2024_07_29stock_company_info2', con=conn)
conn.close()

In [12]:
data

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,산일전기,062040,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","유입, 몰드, 주상, 건식 변압기 등",2024-07-29,12월,박동석,http://www.sanil.co.kr,경기도
1,유가증권,에이치에스효성,487570,기타 금융업,지주사업,2024-07-29,12월,조현상..,http://www.hshyosung.com,서울특별시
2,"코스닥, 투자주의종목",엔에이치스팩31호,481890,금융 지원 서비스업,금융지원서비스업,2024-07-26,12월,이시형,,서울특별시
3,코스닥,SK증권제13호스팩,473950,금융 지원 서비스업,기업인수목적 주식회사,2024-07-25,12월,임율표,,서울특별시
4,코스닥,엑셀세라퓨틱스,373110,기초 의약물질 제조업,CellCor SFD/CD(세포배양배지),2024-07-15,12월,이의일,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
415,코스닥,에이치브이엠,295310,1차 비철금속 제조업,"Ni계 합금, Fe계 합금, 스퍼터링타겟, Cu계 합금 등",2024-06-28,12월,문승호,http://hvm.co.kr/,경기도
416,코넥스,팡스카이,266350,소프트웨어 개발 및 공급업,게임 개발 및 리퍼블리싱,2024-06-28,12월,조호현..,http://www.panggame.com/,서울특별시
417,코스닥,하이젠알앤엠,160190,"전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업","전동기 및 발전기 (범용모터, 서보모터)",2024-06-27,12월,김재학,http://higenrnm.com,경상남도
418,코스닥,한국제15호스팩,479880,금융 지원 서비스업,기업인수합병,2024-06-26,12월,유한,,서울특별시


In [13]:
def str2int(x):
    x = int(x.replace(",", ""))
    return x

In [14]:
stock_price_detail = {}
for idx, code in enumerate(data['종목코드']):
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    r = requests.get(url)
    print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중", end="\r")
    soup = bs(r.text, 'lxml')
    # 가격
    price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
    # 변동금액
    price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
    # 변화율
    rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
    stock_price_detail.setdefault('현재가', []).append(price)
    stock_price_detail.setdefault('변동금액', []).append(price_chage)
    stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
    table = soup.select_one(".no_info")
    for tr in table.select("tr"):
        for td in tr.select('td'):
            stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
stock_price_detail

{'현재가': [47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
  2075,
  5710,
  47850,
  45550,
  2030,
  2075,
  4885,
  2050,
  63800,
  13450,
  16960,
  2720,
  15210,
  939,
  12250,
 

# 수집과 동시에 DB에 저장하기 + 예외처리 하기

In [15]:
today = str(date.today()).replace("-","_")
print(today[:7])

2024_08


In [16]:
str(date.today())

'2024-08-06'

In [17]:
# DB에 접속해서 저장하는 함수
def stock_info_to_db(idx, code, df):
    from sqlalchemy import create_engine
    import pymysql
    from dotenv import load_dotenv
    from datetime import date
    pymysql.install_as_MySQLdb()
    load_dotenv(dotenv_path=".env_db")
    today = str(date.today()).replace("-","_")
    
    engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
    conn = engine.connect()
    df.to_sql(f"{today[:7]}_stock_price_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{today}, {idx}, {code}, {'저장완료':<30s}", end="\r")
    

In [18]:
errors = {}
for idx, (company, code) in enumerate(zip(data['회사명'][:10], data['종목코드'][:10])):
    stock_price_detail = {}
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    try:
        r = requests.get(url)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중                    ", end="\r")
        soup = bs(r.text, 'lxml')
        # 가격
        price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
        # 변동금액
        price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
        # 변화율
        rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
        stock_price_detail.setdefault('수집일',[]).append(str(date.today()))
        stock_price_detail.setdefault('회사명', []).append(company)
        stock_price_detail.setdefault('종목코드', []).append(code)
        stock_price_detail.setdefault('현재가', []).append(price)
        stock_price_detail.setdefault('변동금액', []).append(price_chage)
        stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
        table = soup.select_one(".no_info")
        for tr in table.select("tr"):
            for td in tr.select('td'):
                stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
        df = pd.DataFrame(stock_price_detail)
        stock_info_to_db(idx, code, df)
        time.sleep(5)
    except Exception as e:
        print(e)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중 에러", end="\r")
        errors.setdefault("에러", []).append(code)  # 에러난 코드들 모음
        
stock_price_detail

{'수집일': ['2024-08-06'],
 '회사명': ['  신한글로벌액티브리츠 '],
 '종목코드': ['481850'],
 '현재가': [2720],
 '변동금액': [55],
 '변화율': [2.06],
 '전일': [2665],
 '고가': [2745],
 '거래량': [224973],
 '시가': [2680],
 '저가': [2675],
 '거래대금': [611]}